## 舍入误差

由于计算机内存储小数的精度有限, 因此小数运算很可能出现舍入误差. 

### 小数存储

单精度, 双精度的二进制小数存储与表示. 略.

### 误差估计

定义. ${\rm fl}(x) = x(1+\delta)$, 其中 $|\delta|<\bf u$ 为机器精度. 表示一个实数 $x$ 在计算机内以有限精度 ${\rm}(x)$ 存储.

**Theorem 1** $|\sum_{i=1}^n a_ix_i - {\rm fl}(\sum_{i=1}^n a_ix_i)|<\frac{n\bf u}{1- n\bf u}\sum_{i=1}^n|a_ix_i|$.

### 条件数

可逆方阵 $A$ 的条件数 (condition number) $\kappa$ 定义为 $\kappa = \Vert A\Vert_2 \Vert A^{-1}\Vert_2$. 不可逆方阵条件数为 $+\infty$.


<br>


**Theorem 1**  对于方阵 $A$ , 设最大奇异值和最小奇异值为 $\sigma_1$, $\sigma_n$. 则其条件数  $\kappa = \frac{\sigma_1}{\sigma_n}$.

证: 注意 $\sigma_1 = \Vert A\Vert_2$, $\sigma_n = \Vert A^{-1}\Vert_2$.

据此, 对于一个普通的长方形矩阵, 可定义其条件数为最大奇异值和最小奇异值的比值.

<br>


**Theorem 2** 对于任意方阵 $A$, 总有 $\kappa \geqslant 1$, 取等当且仅当 $A$ 为酉矩阵的倍数(且非零).

证: 由 Theorem 1, 知 $\kappa = \frac{\sigma_1}{\sigma_n}\geqslant 1$. 取等当且仅当 $\sigma_1 = \sigma_n$. 这说明 
$\sigma_1 = \sigma_2 = \dotsc = \sigma_n$, 从而奇异值分解

$$A = 
U\left[\begin{matrix}\sigma_1 & & & \\ \ & \sigma_2 & & \\ \ &  & \ddots & \\ \ & & & \sigma_n
\end{matrix}\right]V=\sigma_1 UV.$$

$$A^*A = \overline \sigma_1\sigma_1 V^*U^*UV=|\sigma_1|^2 I.$$

取 $Q = \frac{A}{|\sigma_1|}$, 则 $Q^*Q = I$ 表明 $Q$ 是酉矩阵.

### 误差与残差

由于舍入误差的存在, 用计算机数值解方程组 $Ax = b$ 可能会带来误差, 即解得 $\hat x=x_*+\delta x$, 其中 $x_*$ 为精确解, $\delta x$ 很小. 

称 $\delta x = \hat x - x_*$ 为向后误差 (backward error) 或误差 (error). 

称 $r = A\hat x - b = A\delta x$ 为向前误差 (forward error) 或残差 (residual).

向前误差充分小时, 向后误差也充分小, 因为

$$\frac{\Vert \delta x\Vert_2}{\Vert x_*\Vert_2} = \frac{\Vert A^{-1}r\Vert_2}{\Vert x_*\Vert_2}
\leqslant \frac{\Vert A^{-1}\Vert_2 \Vert r\Vert_2}{\Vert x_*\Vert_2}
\leqslant \frac{\Vert A^{-1}\Vert_2 \Vert r\Vert_2}{\frac{\Vert Ax_*\Vert_2}{\Vert A\Vert_2}}
=\kappa \frac{\Vert r\Vert_2}{\Vert b\Vert_2}.$$

从上式还能看出, $\kappa$ 越小越好(此时称 $A$ 为良态的), $\kappa$ 太大则称矩阵 $A$ 是病态的.

## 时间复杂度

理论上,

$1 \times n$ 与 $n\times 1$ 的向量相乘(内积), 时间复杂度为 $O(n)$.

$m \times n$ 与 $n\times 1$ 的矩阵-向量相乘, 时间复杂度为 $O(mn)$.

$m \times n$ 与 $n\times l$ 的矩阵-矩阵相乘, 时间复杂度为 $O(mnl)$.

实际上, 在常用的矩阵运算库中, 如 BLAS, 矩阵乘法运算是高度优化的 (如对内存的读取、多线程等), 比自己用循环实现快得多. 也就是说调用一次 $m\times n$ 与 $n\times l$ 的矩阵-矩阵相乘比调用 $l$ 次矩阵-向量相乘效率更高. 

若把向量内积、矩阵-向量乘法、矩阵-矩阵乘法分别称为 1级BLAS、2级BLAS、3级BLAS, 则在设计算法与代码实现中, 应尽量使用少数次高级BLAS**代替**多次低级BLAS.

In [2]:
import numpy as np 
import time
n = 2000
A = np.random.random((n,n)) - 0.5
B = np.random.random((n,n)) - 0.5

clock = time.time()
C = A @ B
print('Level 3 BLAS =',time.time() - clock,'sec')

clock = time.time()
for i in range(n):
    C[:,i] = A @ B[:,i]
print('Level 2 BLAS =',time.time() - clock,'sec')

Level 3 BLAS = 0.209608793258667 sec
Level 2 BLAS = 2.2849934101104736 sec


在实际的大多数基本矩阵问题中, 包括但不限于求最小二乘问题、求舒尔分解、求奇异值分解、解Sylvester方程等, 若输入矩阵的大小为 $n\times n$, 则一般算法复杂度不应超过 $O(n^3)$. 

若输入矩阵的大小为 $m\times n (n\ll m)$, 则一般算法复杂度不应超过 $O(mn^2)$.

这说明这些算法中很少使用完整的$n\times n$矩阵-矩阵乘法.